In [8]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain, RetrievalQA
from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate
import numpy as np

In [3]:
## Read pdfs
loader = PyPDFDirectoryLoader("./data")
docs = loader.load()

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# Split the documents into chunks
text_split = text_splitter.split_documents(docs)

# Initialize the embedding model
embedding_model = OllamaEmbeddings(model="llama3.2:3b")

# Initialize the vector database
vector_db = FAISS.from_documents(text_split, embedding_model)

C:\Users\AHMED ABD ELGWAD\AppData\Local\Temp\ipykernel_5708\3602636715.py:12: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding_model = OllamaEmbeddings(model="llama3.2:3b")


In [5]:
# Embedding using Huggingface
# generating text embeddings
huggingface_embedding = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5", # generating high-quality embeddings for English text
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\AHMED ABD ELGWAD\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\AHMED ABD ELGWAD\.cache\huggingface\hub\models--BAAI--bge-small-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
np.array(huggingface_embedding.embed_query(text_split[0].page_content))


array([-8.46568495e-02, -1.19098574e-02, -3.37892212e-02,  2.94559114e-02,
        5.19159921e-02,  5.73839843e-02, -4.10017632e-02,  2.74267886e-02,
       -1.05128169e-01, -1.58056002e-02,  7.94859007e-02,  5.64318374e-02,
       -1.31764961e-02, -3.41544673e-02,  5.81598748e-03,  4.72547673e-02,
       -1.30746821e-02,  3.12983012e-03, -3.44225504e-02,  3.08406167e-02,
       -4.09085900e-02,  3.52737904e-02, -2.43761521e-02, -4.35831249e-02,
        2.41503548e-02,  1.31986691e-02, -4.84453840e-03,  1.92347337e-02,
       -5.43912724e-02, -1.42735019e-01,  5.15534543e-03,  2.93115471e-02,
       -5.60811162e-02, -8.53530876e-03,  3.14141437e-02,  2.76736487e-02,
       -2.06188187e-02,  8.24231505e-02,  4.15425301e-02,  5.79654723e-02,
       -3.71587574e-02,  6.26160298e-03, -2.41389964e-02, -5.61800878e-03,
       -2.51715519e-02,  5.04970923e-03, -2.52801571e-02, -2.91947275e-03,
       -8.24043527e-03, -5.69604039e-02,  2.30822563e-02, -5.54217771e-03,
        5.11555411e-02,  